# AE Reinforcement learning - Homework 1

## GridWorld

## Acrobot

## Satellite attitude simulator

In [1]:
import attitudeDynamics
%run attitudeDynamics/pss_main.py



Satellite Lat Lon Alt:  -51.48337475911235 90.7177284205493 223.01517125078698 


Satellite Position (normalized): [ 2502.91529395  3271.29548906 -5141.70237969] 


Satellite Position:  [2507.381362560922, 3268.4934320833263, -5141.3076520979175] 

>>> WARNING! Pitch is more than 90 deg. Results may not be accurate
[ 0.65328148 -0.27059805  0.65328148  0.27059805]
[ 0. 90.  0.]
[[ 4.32978028e-17  7.07106781e-01 -7.07106781e-01]
 [-4.32978028e-17  7.07106781e-01  7.07106781e-01]
 [ 1.00000000e+00  0.00000000e+00  6.12323400e-17]] 

[[ 8.32667268e-17  0.00000000e+00 -1.00000000e+00]
 [-7.07106781e-01  7.07106781e-01  0.00000000e+00]
 [ 7.07106781e-01  7.07106781e-01  8.32667268e-17]] 

[[ 4.32978028e-17  4.32978028e-17 -1.00000000e+00]
 [-7.07106781e-01  7.07106781e-01  0.00000000e+00]
 [ 7.07106781e-01  7.07106781e-01  6.12323400e-17]] 

0
1
2
3
4
Elapsed time: 3.368 s


In [1]:
%run attitudeDynamics/attitude_viz.py

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
# %load attitudeDynamics/attitude_viz.py

"""
Created on Wed Sep  5 01:16:54 2018

@author: Vedant
"""

"""
 Simulation of a rotating 3D Cube
 Developed by Leonel Machava <leonelmachava@gmail.com>
 
<a class="vglnk" href="http://codeNtronix.com" rel="nofollow"><span>http</span><span>://</span><span>codeNtronix</span><span>.</span><span>com</span></a>
 
"""
import sys, math, pygame
from operator import itemgetter
 
class Point3D:
    def __init__(self, x = 0, y = 0, z = 0):
        self.x, self.y, self.z = float(x), float(y), float(z)
 
    def rotateX(self, angle):
        """ Rotates the point around the X axis by the given angle in degrees. """
        rad = angle * math.pi / 180
        cosa = math.cos(rad)
        sina = math.sin(rad)
        y = self.y * cosa - self.z * sina
        z = self.y * sina + self.z * cosa
        return Point3D(self.x, y, z)
 
    def rotateY(self, angle):
        """ Rotates the point around the Y axis by the given angle in degrees. """
        rad = angle * math.pi / 180
        cosa = math.cos(rad)
        sina = math.sin(rad)
        z = self.z * cosa - self.x * sina
        x = self.z * sina + self.x * cosa
        return Point3D(x, self.y, z)
 
    def rotateZ(self, angle):
        """ Rotates the point around the Z axis by the given angle in degrees. """
        rad = angle * math.pi / 180
        cosa = math.cos(rad)
        sina = math.sin(rad)
        x = self.x * cosa - self.y * sina
        y = self.x * sina + self.y * cosa
        return Point3D(x, y, self.z)
 
    def project(self, win_width, win_height, fov, viewer_distance):
        """ Transforms this 3D point to 2D using a perspective projection. """
        factor = fov / (viewer_distance + self.z)
        x = self.x * factor + win_width / 2
        y = -self.y * factor + win_height / 2
        return Point3D(x, y, self.z)
 
class Simulation:
    def __init__(self, win_width = 640, win_height = 480):
        pygame.init()
 
        self.screen = pygame.display.set_mode((win_width, win_height))
        pygame.display.set_caption("Simulation of a rotating 3D Cube")
 
        self.clock = pygame.time.Clock()
 
        self.vertices = [
            Point3D(-1,1,-1),
            Point3D(1,1,-1),
            Point3D(1,-1,-1),
            Point3D(-1,-1,-1),
            Point3D(-1,1,1),
            Point3D(1,1,1),
            Point3D(1,-1,1),
            Point3D(-1,-1,1)
        ]
 
        # Define the vertices that compose each of the 6 faces. These numbers are
        # indices to the vertices list defined above.
        self.faces  = [(0,1,2,3),(1,5,6,2),(5,4,7,6),(4,0,3,7),(0,4,5,1),(3,2,6,7)]
 
        # Define colors for each face
        self.colors = [(255,0,255),(255,0,0),(0,255,0),(0,0,255),(0,255,255),(255,255,0)]
 
        self.angle = 0
 
    def run(self):
        """ Main Loop """
        while 1:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
 
            self.clock.tick(50)
            self.screen.fill((0,32,0))
 
            # It will hold transformed vertices.
            t = []
 
            for v in self.vertices:
                # Rotate the point around X axis, then around Y axis, and finally around Z axis.
                r = v.rotateX(self.angle).rotateY(self.angle).rotateZ(self.angle)
                # Transform the point from 3D to 2D
                p = r.project(self.screen.get_width(), self.screen.get_height(), 256, 4)
                # Put the point in the list of transformed vertices
                t.append(p)
 
            # Calculate the average Z values of each face.
            avg_z = []
            i = 0
            for f in self.faces:
                z = (t[f[0]].z + t[f[1]].z + t[f[2]].z + t[f[3]].z) / 4.0
                avg_z.append([i,z])
                i = i + 1
 
            # Draw the faces using the Painter's algorithm:
            # Distant faces are drawn before the closer ones.
            for tmp in sorted(avg_z,key=itemgetter(1),reverse=True):
                face_index = tmp[0]
                f = self.faces[face_index]
                pointlist = [(t[f[0]].x, t[f[0]].y), (t[f[1]].x, t[f[1]].y),
                             (t[f[1]].x, t[f[1]].y), (t[f[2]].x, t[f[2]].y),
                             (t[f[2]].x, t[f[2]].y), (t[f[3]].x, t[f[3]].y),
                             (t[f[3]].x, t[f[3]].y), (t[f[0]].x, t[f[0]].y)]
                pygame.draw.polygon(self.screen,self.colors[face_index],pointlist)
 
            self.angle += 1
 
            pygame.display.flip()
 
if __name__ == "__main__":
    Simulation().run()

SystemExit: 

C:\Users\Vedant\Anaconda3\envs\RL\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
